In [1]:
from rdflib import Graph, Namespace, URIRef
from typing import Optional
from pydantic import BaseModel
from api.src.function.loc.getType import GetType
from pyfuseki import FusekiUpdate
import httpx
from pysolr import Solr

In [2]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

r = au_update.run_sparql('DROP DEFAULT')
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [2]:
upAcervo = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [29]:
solrAcervo = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solrAcervo.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">191</int>\n</lst>\n</response>\n'

In [8]:
from api.src.schemas.authorities.authority import Variant, AdminMetadata, Element
class Uri(BaseModel):
    value: str
    label: str
    base: Optional[str]

class Affiliation(BaseModel):
    organization: Uri
    affiliationStart: Optional[str]
    affiliationEnd: Optional[str]
    
class Agents(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    fullerName: Optional[Element]
    birthDate: Optional[str]
    birthPlace: Optional[str]
    deathDate: Optional[str]
    hasAffiliation: Optional[list[Affiliation]]
    occupation: Optional[list[Uri]]
    fieldOfActivity: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    contributorOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str
    

In [3]:
authority = 'http://id.loc.gov/authorities/names/n79063441'
graph = Graph()
graph.parse(f'{authority}.rdf')
graph.serialize('marquez.ttl')

<Graph identifier=Nea124e0548d148bfa52eaa3f50c90091 (<class 'rdflib.graph.Graph'>)>

In [6]:
# authority = 'http://id.loc.gov/authorities/names/n80002329'
url = f'http://localhost:8000/authorities/import/loc/agents?authority={authority}'
r = httpx.get(url)
r.status_code

200

In [ ]:
request = Agents(**r.json())
request

In [ ]:
from api.src.function.loc.agents.ElementList import GetElementList
from api.src.function.loc.agents.FullerName import GetFullerName
from api.src.function.loc.Uri import GetUri
from api.src.function.loc.agents.Variant import GetVariant
from api.src.function.loc.agents.Date import GetDate
from api.src.function.loc.agents.BirthPlace import GetBirthPlace
from api.src.function.loc.agents.Affiliation import GetAffiliation
from api.src.function.loc.agents.Occuption import GetOccuption

def ParserAgents(graph, authority):
    # Type
    tipo = GetType(graph, authority)
    

    # adminMetadata
    adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]}
    
    obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'}
    
    # ElementList
    obj = GetElementList(graph, authority, obj)
    
    
    # fullerName
    obj = GetFullerName(graph, authority, obj)
    

    # hasCloseExternalAuthority
    obj = GetUri(authority, graph, "hasCloseExternalAuthority", obj)

    # hasExactExternalAuthority
    obj = GetUri(authority, graph, "hasExactExternalAuthority", obj)

    # Variant
    obj = GetVariant(authority, graph, obj)

    # RWO
    token = authority.split("/")[-1]
    rwo = f'http://id.loc.gov/rwo/agents/{token}'
    # BirthDate
    obj = GetDate(rwo, 'birthDate', graph, obj)
    # deathDate
    obj = GetDate(rwo, 'deathDate', graph, obj)
    # BirthPlace
    obj = GetBirthPlace(rwo, graph, obj)
    # Affiliation
    obj = GetAffiliation(rwo, graph, obj)
    # Occuptions
    obj = GetOccuption(rwo, graph, obj)

    response = Agents(**obj)
    
    return response

response = ParserAgents(graph, authority)
response

In [36]:
token = authority.split("/")[-1]
rwo = f'http://id.loc.gov/rwo/agents/{token}'

obj = GetAffiliation(rwo, graph, {})
obj

{}

In [41]:
def GetAffiliation(rwo, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?organization ?labelOrg ?start ?end
    WHERE  {{      	
    <{rwo}> madsrdf:hasAffiliation ?hasAffiliation .
  ?hasAffiliation madsrdf:organization ?organization .
  ?organization madsrdf:authoritativeLabel ?labelOrg .
  OPTIONAL {{ 
  ?hasAffiliation madsrdf:affiliationStart ?start .
    ?hasAffiliation madsrdf:affiliationEnd ?end  }} }}"""
    result = graph.query(q) 
    
    if len(result.bindings) > 0:
        affiliations = list()
        for i in result.bindings:
          # organization
          orgUri = i.get('organization').toPython()
          orgBase = orgUri.split('//')[1].split("/")[0]
          organization = {'value': orgUri, 'label': i.get('labelOrg').toPython(), 'base': orgBase}
          affiliation = {'organization': organization }
          if i.get('start'):
            affiliation['affiliationStart'] = i.get('start').toPython()
          if i.get('end'):
            affiliation['affiliationEnd'] = i.get('end').toPython()  

          affiliations.append(affiliation)
        obj['hasAffiliation'] = affiliations

    return obj

obj = GetAffiliation(rwo, graph, {})
obj

{'hasAffiliation': [{'organization': {'value': 'http://id.loc.gov/authorities/names/n85299803',
    'label': 'University of Iowa. Department of Neurology',
    'base': 'id.loc.gov'}}]}

In [ ]:
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>"""

def MakeGraphAgents(request, id):
    graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            madsrdf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <{request.adminMetadata.assigner}> ;
            bf:creationDate "{request.adminMetadata.creationDate}"^^xsd:date ;
            bf:descriptionLanguage <http://id.loc.gov/vocabulary/languages/eng> ;
            bf:descriptionModifier <{request.adminMetadata.assigner}> ; 
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{request.adminMetadata.generationProcess}" ;
                    bf:generationDate "{request.adminMetadata.generationDate}"^^xsd:dateTime ] ;
            bf:identifiedBy {MakeIdentifier(request.adminMetadata.identifiedBy, id)} ; 
            bf:status {request.adminMetadata.status.value} ] ; 
            madsrdf:authoritativeLabel "{MakeLabel(request.elementList)}" ;
            madsrdf:elementList ( {MakeElement(request.elementList)} ) ; 
            { FullerName(request) if request.fullerName else ''  }
            { f'madsrdf:birthDate "{request.birthDate}" ;' if request.birthDate else ''  }
            { BirthPlace(request) if request.birthPlace else ''  }
            { f'madsrdf:deathDate "{request.deathDate}" ;' if request.deathDate else ''  }
            { f'madsrdf:occupation {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            { f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            { f'madsrdf:hasExactExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasExactExternalAuthority])} ;' if request.hasExactExternalAuthority else ''}
            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .          
            }} 
        }}"""
    return graph

response = MakeGraphAgents(request, id)
print(response)

SOLR

In [ ]:
birthDate: Optional[str]
    birthPlace: Optional[str]
    deathDate: Optional[str]
    hasAffiliation: Optional[list[Affiliation]]
    occupation: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    contributorOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

In [51]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
uri = 'https://bibliokeia.com/authorities/PersonalName/bkau-1'
uriID = uri.split("/")[-1]
r = solr.search(q=f'id:{uriID}', **{'fl': '*,[child]'})
[doc] = r.docs
ids = [uriID]

In [133]:
def MakeDocAgents(request, id):

    doc = {
            'id': id,
            'type': request.type,
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": f'{MakeLabel(request.elementList)}' ,
            "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
    
    if request.fullerName:
        doc['fullerName'] = request.fullerName.elementValue.value
    if request.birthDate:
        doc['birthDate'] = request.birthDate
    if request.birthPlace:
        doc['birthPlace'] = request.birthPlace
    if request.deathDate:
        doc['deathDate'] = request.deathDate
    
    # hasAffiliation
    
    if request.hasAffiliation:
        affiliations = list()
        for i in request.hasAffiliation:
            a = {
                'id': f"{id}/hasAffiliation#{i.organization.value.split('/')[-1]}",
                'organization': i.organization.label,
                'affiliationStart': i.affiliationStart,
            }
            if i.affiliationEnd:
                a['affiliationEnd'] = i.affiliationEnd
            affiliations.append(a)
        doc['hasAffiliation'] = affiliations
    
    if request.hasVariant:
        variants = list()
        for i in request.hasVariant:
            label = [j.elementValue.value for j in i.elementList]
            label = " ".join(label)
            variants.append(label)
        doc['variant'] = variants

    # hasCloseExternalAuthority
    uris = list()
    for i in request.hasCloseExternalAuthority:
        uri = {
                'id': f"{id}/hasCloseExternalAuthority#{i.value.split('/')[-1]}",
                'uri': i.value, 
                'label': i.label, 
                'base': i.base }
        if i.base == 'www.wikidata.org':
            imagem = GetImagem(i.value)
            if imagem:
                doc['imagem'] = imagem

    uris.append(uri)
    doc['hasCloseExternalAuthority'] = uris

    return doc

doc = MakeDocAgents(request, id)

In [15]:
class Label(BaseModel):
    value: str
    lang: Optional[str]

class Mads(BaseModel):
    metadata: str
    type: Optional[str]
    label: Label

class EditAuthority(BaseModel):
    action: str
    data: Mads

obj = {
  "action": "remove",
  "data": {
    "metadata": "fullerName",
    "type": "PersonalName",
    "label": {
      "value": "Gabriel José de la Concordia"
    }
  }
}

au_update = FusekiUpdate('http://localhost:3030', 'authorities')
authority = 'https://bibliokeia.com/authorities/PersonalName/bkau-1'
request = EditAuthority(**obj)


def DeleteDataJena(authority, request):

    delete = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                 DELETE DATA
                {{ GRAPH <{authority}> 
                {{ <{authority}>  madsrdf:{request.data.metadata}  "{request.data.value}" }} }} ;"""
    response = au_update.run_sparql(delete)
    return response.convert()

# response = DeleteDataJena(authority, request)


In [26]:
request.data.label.value

'Gabriel José de la Concordia'

In [22]:
delete = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
            WITH <{authority}> 
            DELETE {{ <{authority}>  madsrdf:{request.data.metadata}  ?node .
                ?node rdf:type ?type .
                ?node rdfs:label "{request.data.label.value}" }}
            WHERE {{ <{authority}>  madsrdf:{request.data.metadata}  ?node .
                ?node rdf:type ?type .
                ?node rdfs:label "{request.data.label.value}"  }}"""



response = au_update.run_sparql(delete)
response.convert()


{'statusCode': 200, 'message': 'Update succeeded'}

In [23]:
request.data.label.value

'Gabriel José de la Concordia'

In [24]:
add = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
INSERT DATA
{{ GRAPH <{authority}> 
{{ <{authority}>  madsrdf:{request.data.metadata} [ a madsrdf:{request.data.type} ;
            rdfs:label "{request.data.label.value}" ] }} }}"""

response = au_update.run_sparql(add)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [ ]:
id = authority.split("/")[-1] 
doc = {'id': id,
       f'{request.data.metadata}': {'set':request.data.label.value }
       }
    response = solrAu.add([doc], commit=True)

In [27]:
id = authority.split("/")[-1] 
doc = {'id': id,
       f'{request.data.metadata}': {'set':request.data.label.value }
       }

solrAu = Solr('http://localhost:8983/solr/authorities/', timeout=10)
response = solrAu.add([doc], commit=True)
response

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":123}}\n'

In [ ]:
obj = {
  "action": "add",
  "data": {
    "metadata": "birthDate",
    "value": "Rouen (France)"
  }
}

delete = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                 DELETE DATA
                {{ GRAPH <{authority}> 
                {{ <{authority}>  madsrdf:{request.data.metadata}  "{request.data.label.value}" }} }} ;"""  

In [28]:
request.data.label.value

'Gabriel José de la Concordia'